# Quelques utilisations avancées des LED

Le [premier cahier](https://github.com/g-vidal/CahierDeProgrammes/blob/master/JeuxDeLumiere/simpleLED.ipynb) a permis de se familiariser avec le pilotage à distance par des programmes et une interface web de diodes électroluminescentes. Celui-ci aborde quelques points plus avancés tirant profit de la polarité des LEDs ou permettant de compenser l'intensité du courant insuffisante pour allumer les LEDs

## Utilisation de transistors pour augmenter l'intensité traversant une LED

Le objets connectés comme l'Edison ou la Raspberry Pi sont conçus pour être peu gourmands en électricité ce qui se traduit par des intensités plus faibles délivrées par les broches du GPIO. On se retrouve confrontés au problème inverse de celui qui faisait "griller" les LEDs parce que l'intensité délivrée par les micro-contrôleurs était trop forte. Comme il fallait ajouter un nouveau composant de protection (une résistance) il faut ajouter ici un composant supplémentaire pour rehausser l'intensité et bien sur apporter une source de courant externe. le montage utilisé est encadré sur l'image ci_dessous:

![Alimentation de deux LEDs avec transistor](images/ledboard4.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)

Une alimentation 1.8V ou 3.3V (suivant la tension disponible sur l'IoT concerné) est branchée sur les fils rouge et noir, les fils de modulation broche 32 et 33 selon la même nomenclature que le premier cahier dans la continuité des deux résistances extérieiures. Le code ci-dessous permet de contrôler l'allumage et l'extincton des diodes (Leds) rouge et verte.
* importation des programmes de la bibliothèque mraa
* déclaration de l'utilisation des broches 32 et 33 GPIO12 et GPIO13

In [1]:
import mraa, time
ledg = mraa.Gpio(32)
ledr = mraa.Gpio(33)

* Choix du sens de travail de la broche *out* pour envoyer une intensité dans le circuit

In [2]:
ledg.dir(mraa.DIR_OUT)
ledr.dir(mraa.DIR_OUT)

0

Mise en place du clignotement pendant 1 minute avec une fréquence d'un clignotement par seconde pour l'une et de 2 clignotements par seconde pour l'autre

In [3]:
for i in range(0,240,1) :
    j = 2 
    k = 4 
    if (i % k == 0):
        if ((i / k) % 2 == 0):
            ledr.write(1)
        else :
            ledr.write(0)
    if (i % j == 0):
        ledg.write(1)
        time.sleep(0.25)
    else :
        ledg.write(0)
        time.sleep(0.25)


### Exercice 

Modifier l'alimentation des transistors  et observer l'effet sur les diodes (Leds). Choisir d'autres fréquences d'échantillonnage, décrire le clignotement attendu, écrire le programme correspondant

In [ ]:
for i in range(-,-,-) :
    j = - 
    k = - 
    if (i % k == -):
        
    else :


## Clignotement utilisant deux broches de contrôle et pas de masse

Comme nous l'avons vu dans les différents programmes les broches du GPIO que nous avons utilisées ont deux positions possibles soit elles envoient du courant soit leur potentiel est à 0. Si l'on connecte une diode (Led) à deux broches l'une au maximum (1) l'autre à 0 la diode (Led) va être parcourue par un courant et s'allumer. Si on permute la valeur des deux broches (Leds) la diode s'éteint mais si on permute le branchement elle se rallume. Nous allons utiliser le montage encadré ci'dessous où deux diodes (Leds) sont insérées dans la plaque en parallèle et l'une permutée par rapport à l'autre. 

![Alimentation de deux LEDs par deux broches](images/ledboard3.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Il est possible de représenter la situation sur un dessin illustrant le branchement et l'activité des diodes:

![Schéma de deux LEDs dont le branchement est permutté](images/altern2.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Nous utilisons les broches 32 et 33 l'une branchée sur le fil commun l'autre dans la continuité de la résistance. Noter qu'**il est impossible d'allumer simultanément les deux diodes**.
On utilise les déclarations précédentes des leds rouge et verte ler ledg, on met en place un clignotement symétrique de 1/10ème de seconde pour la verte et asymétrique 2/10ème-3/10ème pour la rouge.


In [5]:
for i in range(0,90,1) :
    ledr.write(0)
    for j in range(4) :
        ledg.write(0)
        time.sleep(0.1)
        ledg.write(1)
        time.sleep(0.1)
    ledg.write(0)
    for j in range(3) :
        ledr.write(1)
        time.sleep(0.3)
        ledr.write(0)
        time.sleep(0.2)

### exercice 

Mettre en place un clignotement symétrique simple des diodes (LEDs) rouge/verte. Décrire un clignotement plus complexe et le programmer.

In [ ]:
for i in range(-,-,-) :

## Comment contrôler 6 LED avec 3 fils de commande seulement ?

Dans la vie courante on est habitué à contrôler une ampoule avec deux fils, dans les exercices effectués nous avons aussi utilisé un seul fil de contrôle par LEDs et un fil de masse éventuellement commun à plusieurs LED. Dans l'exemple précédent on a vu qu'en gbranchant les deux LEDs symétriquement il était possible en permuttant la valeur des broches d'obtenir un clignotement. Il existe des dispositifs présentant de nombreuses LEDs contrôlées individuellement ou par groupes, il s'agit par exemple des chenillards lumineux dans les espaces ludiques ou des guirlandes de Noël. Malgré le grand nombre de LEDs ces dispositifs sont contrôlés par un très faible nombre de fils, comment cela est-il possible ?

Nous allons aborder un exemple basique permettant de contrôler 6 LED avec 3 fils, le procédé utilisé est le Charlie-Plexing. Ce procédé permet de contrôler avec n fils n * (n-1) LEDs; soit pour nous 3 x 2 = 6.
On utilise pour cette partie le montage encadré en rouge ci-dessous.

![CharliePlexing à 6 LED](images/ledboard6.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Les fils gris représentent les points de branchement des 3 fils de contrôle, le cablage est représenté par les fils rouges. Ce procédé repose sur la possibilité de mettre la broche non pas dans deux mais dans 3 états : 
 * état masse ..............valeur out 0
 * état alimentation........valeur out 1
 * état résistance infinie .valeur in Z
 
Le Schéma de principe du fonctionnement est donné dans l'image suivante :

![Schéma de principe du CharliePlexing à 6 LED](images/charlie6.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)               

Nous représenterons chaque état par la séquence d'abbréviations la caractérisant dans l'ordre des broches 1-2-3. Par exemple 0Z1 signifie que le + est sur la broche 3 que le - est sur la broche 1 et que la broche 2 est bloquée. On a donc les états suivants
 * 10Z
 * 1Z0
 * 01Z
 * 0Z1
 * Z10
 * Z01
Cela fournit bien 6 états possibles pour notre circuit à 3 fils. Nous allons **définir** (instruction *def*) 6 ensembles d'instructions permettant chacun de réaliser l'état attendu ce sont des *fonctions* que nous nommerons du nom de l'état du circuit qu'elles provoquent. Nous en produirons une 7ème qui assurera l'enchaînement des états blink6.

In [11]:
import mraa, time

In [12]:
led1 = mraa.Gpio(23)
led2 = mraa.Gpio(32)
led3 = mraa.Gpio(33)
dur = 0.05

In [15]:
def etat_10Z(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_IN)
    led1.write(1)
    led2.write(0)
    time.sleep(dur)
    return

def etat_1Z0(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_OUT)
    led1.write(1)
    led3.write(0)
    time.sleep(dur)
    return

def etat_01Z(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_IN)
    led1.write(0)
    led2.write(1)
    time.sleep(dur)
    return

def etat_0Z1(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_OUT)
    led1.write(0)
    led3.write(1)
    time.sleep(dur)
    return

def etat_Z10(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_OUT)
    led2.write(1)
    led3.write(0)
    time.sleep(dur)
    return

def etat_Z01(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_OUT)
    led2.write(0)
    led3.write(1)
    time.sleep(dur)
    return

def etat_000(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_IN)
    return


def blink6(leda,ledb,ledc) :
    for i in range(20) :
        etat_01Z(leda,ledb,ledc)
        etat_10Z(leda,ledb,ledc)
        etat_0Z1(leda,ledb,ledc)
        etat_1Z0(leda,ledb,ledc)
        etat_Z01(leda,ledb,ledc)     
        etat_Z10(leda,ledb,ledc)
        etat_000(leda,ledb,ledc)
        etat_Z10(leda,ledb,ledc)
        etat_Z01(leda,ledb,ledc)
        etat_1Z0(leda,ledb,ledc)
        etat_0Z1(leda,ledb,ledc)
        etat_10Z(leda,ledb,ledc)
        etat_01Z(leda,ledb,ledc)
    etat_000(leda,ledb,ledc)
    return

blink6(led1,led2,led3)

### Exercices

- Modifier la fréquence de balayage 
- Faire un balayage dans un seul sens
- Changer le mode de balayage (par exemple +2 -1)

## Utilisation d'une LED RGB

Plusieurs types de LED RGB existent dans le commerce, certaines ont une *cathode commune* d'autres ont une *anode commune*. Les branchements sont différents et il est important de bien comprendre quel sera le sens du courant et quelle sera la tension aux différentes bornes avant de faire les branchements sous peine de **détruire** la nano machine utilsée. Les branchements pour Arduino (5v) que l'on trouve sur le web ne conviennent pas forcément pour un Edison (1.8V) ou un Raspberry PI (3.3V). Une des solutions pour éviter les catastrophes est d'utiliser un transistor (qui impose une réflexion plus avancée sur les branchements à réaliser et bien sûr une maîtrise de la puissance fournie). 

Dans ce qui suit nous utilisons une LED RGB à **cathode commune** et nous proposons un branchement direct utilisant une source 3.3V. Des résistances permettent d'harmoniser les tensions.

![Montage d'une LED RGB à cathode commune](images/ledboard5.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
La terre est branchée sur le fil noir les trois connecteurs GPIO 22 23 24  sont branchés respectivement dans les trous 24 26 et 28 de la plaque d'essai.
Le programme ci dessous :
 * allume la diode rouge pendant 0.25s à 15% de la puissance max (fréquence 200Hz)
 * augmente la puissance jusqu'à 80% par pas de 5% et intervalle de 0.25s
 * fait clignoter la LED à une fréquence de 8Hz Rouge 15 Hz Vert et 18 Hz bleu
 

In [33]:
import RPi.GPIO as gpio, time
gpio.setmode(gpio.BOARD)  # choisir BCM ou BOARD pour identifier les broches. Nous avons choisi BOARD

dur = 0.25
gpio.setup(15, gpio.OUT)# GPIO 22 broche 15 output  
gpio.setup(16, gpio.OUT)# GPIO 23 broche 16 output 
gpio.setup(18, gpio.OUT)# GPIO 24 broche 18 output  


In [34]:
  
pr = gpio.PWM(18, 200)    # led rouge crée un objet pr  PWM sur le port GPIO22 à 200 Hertz  T=5ms 
pg = gpio.PWM(15, 200)    # led verte crée un objet pg  PWM sur le port GPIO22 à 200 Hertz  T=5ms 
pb = gpio.PWM(16, 200)    # led bleue crée un objet pb  PWM sur le port GPIO22 à 200 Hertz  T=5ms 

# Allumage successif des 3 LEDs et changement d'intensité  
pr.start(15)             # allumage à 15 % (15% duty cycle) 
time.sleep(dur)
for i in range(15,80,5) :
    pr.ChangeDutyCycle(i)   # passage à 80% par pas de 5%
    time.sleep(dur)
pr.ChangeFrequency(8)   # passage de la fréquence à 8 Hz  
time.sleep(10*dur)
pr.stop()
pg.start(15)             # allumage à 15 % (15% duty cycle)   
time.sleep(dur)
for i in range(15,80,5) :
    pg.ChangeDutyCycle(i)   # passage à 80% par pas de 5% 
    time.sleep(dur)
pg.ChangeFrequency(15)   # passage de la fréquence à 15 Hz  
time.sleep(10*dur)
pg.stop()
pb.start(15)             # allumage à 15 % (15% duty cycle)   
time.sleep(dur)
for i in range(15,80,5) :
    pb.ChangeDutyCycle(i)   # passage à 80% par pas de 5%
    time.sleep(dur)
pb.ChangeFrequency(18)   # passage de la fréquence à 18 Hz  
time.sleep(10*dur)
pb.stop()

In [30]:
gpio.cleanup()          #nettoyage de l'état du gpio  